# Laplace-Beltrami operator

This worksheet explores the issue raised by this [ask.sagemath question](https://ask.sagemath.org/question/37513/better-implementation-for-laplace-beltrami-operator/)

In [1]:
version()

'SageMath version 8.0.beta4, Release Date: 2017-04-27'

In [2]:
%display latex

In [3]:
Parallelism().set(nproc=1)

## Manifold and coordinate charts

In [4]:
M = Manifold(2*3,'R^6',field='real',start_index=1)

In [5]:
m1, m2 = var('m1 m2', domain='real')

In [6]:
m_CM = m1+m2; mu1 = m1/m_CM; mu2 = m2/m_CM; mu = m1*m2/m_CM

In [7]:
c_Cart.<x1,y1,z1,x2,y2,z2> = M.chart()
c_Cart

Chart (R^6, (x1, y1, z1, x2, y2, z2))

In [8]:
g = M.riemannian_metric('g')
g[1,1],g[2,2],g[3,3], g[4,4],g[5,5],g[6,6] = m1,m1,m1, m2,m2,m2; 
g.display()

g = m1 dx1*dx1 + m1 dy1*dy1 + m1 dz1*dz1 + m2 dx2*dx2 + m2 dy2*dy2 + m2 dz2*dz2

In [9]:
c_CM.<X,Y,Z,x,y,z> = M.chart()
c_CM

Chart (R^6, (X, Y, Z, x, y, z))

In [10]:
ch_Cart_CM = c_Cart.transition_map(c_CM, [mu1*x1+mu2*x2,mu1*y1+mu2*y2,mu1*z1+mu2*z2, x1-x2,y1-y2,z1-z2],
                                   restrictions2 = [x!=0, y!=0, z!=0])
ch_Cart_CM.display()

X = m1*x1/(m1 + m2) + m2*x2/(m1 + m2)
Y = m1*y1/(m1 + m2) + m2*y2/(m1 + m2)
Z = m1*z1/(m1 + m2) + m2*z2/(m1 + m2)
x = x1 - x2
y = y1 - y2
z = z1 - z2

In [11]:
ch_Cart_CM.inverse().display()

x1 = (X*(m1 + m2) + m2*x)/(m1 + m2)
y1 = (Y*m1 + (Y + y)*m2)/(m1 + m2)
z1 = (Z*m1 + (Z + z)*m2)/(m1 + m2)
x2 = (X*(m1 + m2) - m1*x)/(m1 + m2)
y2 = ((Y - y)*m1 + Y*m2)/(m1 + m2)
z2 = ((Z - z)*m1 + Z*m2)/(m1 + m2)

In [12]:
M.set_default_chart(c_CM)
M.set_default_frame(c_CM.frame())

## The scalar field $\psi$

In [13]:
psiX = M.scalar_field({c_CM: function('Psi_X')(X,Y,Z)}, name='psiX',
                      latex_name='\Psi_X')
psiX.display()

psiX: R^6 --> R
   (x1, y1, z1, x2, y2, z2) |--> Psi_X((m1*x1 + m2*x2)/(m1 + m2), (m1*y1 + m2*y2)/(m1 + m2), (m1*z1 + m2*z2)/(m1 + m2))
   (X, Y, Z, x, y, z) |--> Psi_X(X, Y, Z)

In [14]:
psix = M.scalar_field({c_CM: function('psi_x')(x,y,z)}, name='psix',
                      latex_name='\psi_x')
psix.display()

psix: R^6 --> R
   (x1, y1, z1, x2, y2, z2) |--> psi_x(x1 - x2, y1 - y2, z1 - z2)
   (X, Y, Z, x, y, z) |--> psi_x(x, y, z)

In [15]:
psi = psiX*psix
psi.set_name('psi', latex_name='\psi')
psi.display()

psi: R^6 --> R
   (x1, y1, z1, x2, y2, z2) |--> Psi_X((m1*x1 + m2*x2)/(m1 + m2), (m1*y1 + m2*y2)/(m1 + m2), (m1*z1 + m2*z2)/(m1 + m2))*psi_x(x1 - x2, y1 - y2, z1 - z2)
   (X, Y, Z, x, y, z) |--> Psi_X(X, Y, Z)*psi_x(x, y, z)

## Laplace-Beltrami operator as $*\mathrm{d}*\mathrm{d}\psi$

In [16]:
%time dpsi = psi.exterior_derivative()
print(dpsi)

CPU times: user 1.8 s, sys: 72 ms, total: 1.87 s
Wall time: 1.81 s
1-form dpsi on the 6-dimensional differentiable manifold R^6


In [17]:
dpsi.display()

dpsi = psi_x(x, y, z)*d(Psi_X)/dX dX + psi_x(x, y, z)*d(Psi_X)/dY dY + psi_x(x, y, z)*d(Psi_X)/dZ dZ + Psi_X(X, Y, Z)*d(psi_x)/dx dx + Psi_X(X, Y, Z)*d(psi_x)/dy dy + Psi_X(X, Y, Z)*d(psi_x)/dz dz

In [18]:
%time sdpsi = dpsi.hodge_dual(g)
print(sdpsi)

CPU times: user 3min 21s, sys: 3.2 s, total: 3min 24s
Wall time: 3min 23s
5-form *dpsi on the 6-dimensional differentiable manifold R^6


In [19]:
%time dsdpsi = sdpsi.exterior_derivative()
print(dsdpsi)

CPU times: user 4.1 s, sys: 140 ms, total: 4.24 s
Wall time: 4.09 s
6-form d*dpsi on the 6-dimensional differentiable manifold R^6


In [20]:
dsdpsi.display()

d*dpsi = (m1*m2*(d^2(Psi_X)/dX^2 + d^2(Psi_X)/dY^2 + d^2(Psi_X)/dZ^2)*psi_x(x, y, z) + (m1^2*Psi_X(X, Y, Z) + 2*m1*m2*Psi_X(X, Y, Z) + m2^2*Psi_X(X, Y, Z))*d^2(psi_x)/dx^2 + (m1^2*Psi_X(X, Y, Z) + 2*m1*m2*Psi_X(X, Y, Z) + m2^2*Psi_X(X, Y, Z))*d^2(psi_x)/dy^2 + (m1^2*Psi_X(X, Y, Z) + 2*m1*m2*Psi_X(X, Y, Z) + m2^2*Psi_X(X, Y, Z))*d^2(psi_x)/dz^2)*sqrt(m1)*sqrt(m2)/(m1 + m2) dX/\dY/\dZ/\dx/\dy/\dz

In [21]:
%time LBpsi = dsdpsi.hodge_dual(g)
print(LBpsi)

CPU times: user 7min 11s, sys: 3.55 s, total: 7min 15s
Wall time: 7min 11s
Scalar field *d*dpsi on the 6-dimensional differentiable manifold R^6


## Laplace-Beltrami operator as $\nabla_\mu \nabla^\mu \psi$

In [22]:
nabla = g.connection()
print(nabla)

Levi-Civita connection nabla_g associated with the Riemannian metric g on the 6-dimensional differentiable manifold R^6


In [23]:
%time LBpsi1 = nabla(nabla(psi).up(g)).trace()
LBpsi1

CPU times: user 12.7 s, sys: 248 ms, total: 13 s
Wall time: 12.8 s


Scalar field on the 6-dimensional differentiable manifold R^6

In [24]:
LBpsi1.coord_function()

(m1*m2*(d^2(Psi_X)/dX^2 + d^2(Psi_X)/dY^2 + d^2(Psi_X)/dZ^2)*psi_x(x, y, z) + (m1^2*Psi_X(X, Y, Z) + 2*m1*m2*Psi_X(X, Y, Z) + m2^2*Psi_X(X, Y, Z))*d^2(psi_x)/dx^2 + (m1^2*Psi_X(X, Y, Z) + 2*m1*m2*Psi_X(X, Y, Z) + m2^2*Psi_X(X, Y, Z))*d^2(psi_x)/dy^2 + (m1^2*Psi_X(X, Y, Z) + 2*m1*m2*Psi_X(X, Y, Z) + m2^2*Psi_X(X, Y, Z))*d^2(psi_x)/dz^2)/(m1^2*m2 + m1*m2^2)

Test:

In [25]:
(LBpsi1 - LBpsi).display()

zero: R^6 --> R
   (x1, y1, z1, x2, y2, z2) |--> 0
   (X, Y, Z, x, y, z) |--> 0